In [1]:
import sqlite3
import json
import os
from io import BytesIO
from os import getenv
from dotenv import load_dotenv
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.keys import Keys
import pytesseract
from PIL import Image
import cv2
import numpy as np
from re import split
import csv

In [2]:
load_dotenv()
DB_PATH = getenv('DB_PATH')

In [ ]:
con = sqlite3.connect("mangakakalot.db")
cur = con.cursor()

for row in cur.execute("SELECT * FROM Manga"):
    title, url, current, latest = row

    current = current.split(":")[0].split(" ")[-1]
    if current == "" or current == " ":
        current = 0
    elif "." in current:
        current = float(current)
    else:
        current = int(current)

    latest = latest.split(":")[0].split(" ")[-1]
    if latest == "" or latest == " ":
        latest = 0
    elif "." in latest:
        latest = float(latest)
    else:
        latest = int(latest)

In [ ]:
test = "132"
if "." in test:
    print(True)

In [ ]:
def func(line):
    line = line.strip('"')
    curr_chap, latest_chap = "", ""

    if line[0] == "C": # it starts with "Chapter"
        line = line.split("Chapter")[1:]
        first = line[0].split("Vol")

        if len(first) == 1:
            curr_chap = "Chapter" + line[0][:-1]
            latest_chap = "Chapter " + line[1].strip()
        elif len(first) == 2:
            curr_chap = "Chapter" + first[0][:-1]
            latest_chap = "Vol" + first[1] + "Chapter " + line[-1].strip()

    else: # it starts with "Vol"
        line = line.split("Vol")[1:]

        if len(line) == 1: # the latest chapter starts with "Chapter"
            line = line[0].rsplit(",Chapter", 1)
            curr_chap = "Vol" + line[0]
            latest_chap = "Chapter " + line[-1].strip()
        elif len(line) == 2: # the latest chapter starts with "Vol"
            curr_chap = "Vol" + line[0][:-1]
            latest_chap = "Vol" + line[1].strip()

        # print(f"{curr_chap} --- {latest_chap}")

    return curr_chap, latest_chap

one = "Vol.5 Chapter 43: Cheers,Vol.15 Chapter 165: Takehi Gouda's Ability\n"
two = "Chapter 3,Chapter 11\n"
three = "Chapter 14,Vol.4 Chapter 18\n"
four = "Vol.4 Chapter 14,Chapter 18: Something else is going on\n"

print(func(four))

In [ ]:
titles = []
urls = []
current = []
latest = []
full = []

with open("tempdb.csv", "r", encoding="utf-8") as file:
    csvwriter = csv.reader(file)
    for i, lines in enumerate(file):
        full.append(lines)
        if i == 0:
            continue

        lines = split("https", lines)
        titles.append(lines[0][:-1].strip('"'))

        lines = lines[1].split(",", maxsplit=1)
        urls.append(f"https{lines[0]}")
        
        curr_chap, latest_chap = func(lines[1])
        current.append(curr_chap)
        latest.append(latest_chap)

In [ ]:
driver = webdriver.Chrome("..\..\web_scraping\chromedriver-win64\chromedriver.exe")
driver.get("https://comick.io/home")

In [ ]:
title = titles[0]
search_form = driver.find_element_by_xpath("//form[contains(@itemtype, 'http://schema.org/SearchAction')]")
search_bar = search_form.find_element_by_xpath(".//input")
search_button = search_form.find_element_by_xpath(".//button")
search_bar.send_keys(title)
search_button.click()

In [ ]:
results_name = driver.find_elements_by_xpath(".//div[contains(@class, 'tt')]")[:5]
results_img = driver.find_elements_by_xpath(".//div[contains(@class, 'limit')]/img")
results_img = [r.get_attribute("src") for r in results_img]

In [ ]:
all_names = {}

for title in titles:
    search_form = driver.find_element_by_xpath("//form[contains(@itemtype, 'http://schema.org/SearchAction')]")
    search_bar = search_form.find_element_by_xpath(".//input")
    search_button = search_form.find_element_by_xpath(".//button")
    search_bar.send_keys(title)
    search_button.click()

    names = driver.find_elements_by_xpath(".//div[contains(@class, 'tt')]")[:5]
    names = [name.text for name in names]
    all_names[title] = names

for title, results in all_names.items():
    if len(results) > 0:
        print(f"{title}: {results}")

In [ ]:
images_path = "D:/Vishals Folder/Study/Code/Python/manga_project/images/full"
os.listdir(images_path)

In [ ]:
def img_to_bin(path):
    with open(path, 'rb') as file:
        data = file.read()
    return data

con = sqlite3.connect(DB_PATH)
cur = con.cursor()

complete = 0
count = 0
images_path = "D:/Vishals Folder/Study/Code/Python/manga_project/images/full/"
image_names = os.listdir(images_path)
total = 0

cur.execute("SELECT * FROM Manga")
for row in cur.fetchall():
    title, *other = row

    not_allowed = ['?', '/', '%']
    change = {'’': "'", '“': '"', '”': '"'}
    remove = ['»', '«']
    
    img_title = "".join([i for i in title if i not in not_allowed])
    new_title = ''

    for char in img_title:
        if char in change:
            new_title = new_title + change[char]
            continue
        if char in remove:
            continue

        new_title = new_title + char
    img_title = new_title

    if f"{img_title}.jpg" not in image_names:
        print(f"{img_title}, {title}")
        continue

    image = img_to_bin(images_path + img_title + ".jpg")
    cur.execute("UPDATE Manga set image=? where title=?", (image, title))
    
# con.commit()

cur.close()
con.close()

In [ ]:
def bin_image(path):
    with open(path, 'rb') as file:
        data = file.read()
    return data

# image = "D:/Vishals Folder/Study/Code/Python/manga_project/images/full/Bartender.jpg"
# bin_image(image)
change = {'’': "'", '“': '"', '”': '"'}
remove = ['»', '«']

con = sqlite3.connect(DB_PATH)
cur = con.cursor()

# print(len(cur.execute("SELECT * FROM Manga").fetchall()))
count = 0
cur.execute("SELECT * FROM Manga")

for row in cur.fetchall():
    count += 1
    title, *other = row
    print(title)
    new_title = ''

    for char in title:
        if char in change:
            new_title = new_title + change[char]
            continue
        if char in remove:
            continue

        new_title = new_title + char

    if new_title != title:
        cur.execute("UPDATE Manga set title=? where title=?", (new_title, title))
        
con.commit()

print(count)
cur.close()
con.close()

In [6]:
title = "I'm a D-Rank Adventurer, for Some Reason I Got Recruited Into a Hero Party, and Now the Princess Is Stalking Me"
a = "I'm a D-Rank Adventurer, for Some Reason I Got Recruited Into a Hero Party, and Now the Princess Is Stalking Me"

if a == title:
    print('huh?')

con = sqlite3.connect(DB_PATH)
cur = con.cursor()

image = cur.execute("SELECT * from Manga where title=?", (title,)).fetchone()
print(image)
cur.close()
con.close()

huh?
("I'm a D-Rank Adventurer, for Some Reason I Got Recruited Into a Hero Party, and Now the Princess Is Stalking Me", '', 27, 28, 'lngqm7OZ', 'comick', 'If you become a hero and defeat the Demon King, a life of idleness awaits you...! This is what the young man Jirei believed, and he worked hard day and night to achieve his goal. However, when he found out that the heroes who defeated the Demon King were the ones who were doing all the hard work, he ran away. But the girls that Jirei had been saving on his journeys before he knew it came pouring in one after another to win his heart and body...?', None)


In [162]:
change = {'’': "'", '“': '"', '”': '"'}
remove = ['»', '«']

title = "The Extra’s Academy Survival Guide"
new_title = ''

for char in title:
    if char in change:
        new_title = new_title + change[char]
        continue
    if char in remove:
        continue

    new_title = new_title + char

if new_title != title:
    print(new_title, title)

The Extra's Academy Survival Guide The Extra’s Academy Survival Guide


In [7]:
con = sqlite3.connect(DB_PATH)
cur = con.cursor()

cur.execute("DELETE FROM Manga WHERE title=?", ("The Greatest Estate Designer",))
con.commit()

print(len(cur.execute("SELECT * FROM Manga").fetchall()))

cur.close()
con.close()

98
